<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220405_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 3.8 MB 82.6 MB/s 
     |████████████████████████████████| 212 kB 88.4 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 77.1 MB/s 
     |████████████████████████████████| 136 kB 88.9 MB/s 
     |████████████████████████████████| 127 kB 90.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 79.9 MB/s 
     |████████████████████████████████| 271 kB 91.4 MB/s 
     |████████████████████████████████| 6.5 MB 75.2 MB/s 
     |████████████████████████████████| 596 kB 80.0 MB/s 
     |████████████████████████████████| 895 kB 77.1 MB/s 
     |████████████████████████████████| 1.2 MB 61.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path

In [50]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [51]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [52]:
df = df.fillna('')

In [53]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [54]:
train = df[['text', 'target']]
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...
999995,제품입고 워싱 청바지워싱,C 13 134
999996,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,사업장에서 일반고객을대상으로 필라테스,P 85 856


# 토크나이저, 함수

In [10]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [11]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [55]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,0
1,상점내에서 일반인을 대상으로 채소.과일판매,1
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,영업점에서 일반소비자에게 열쇠잠금장치,3
4,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...
999995,제품입고 워싱 청바지워싱,103
999996,현장에서 고객의요청에의해 실내인테리어,68
999997,영업점에서 일반소비자에게 여성의류 판매,11
999998,사업장에서 일반고객을대상으로 필라테스,12


In [56]:
my_dict['  '] = len(my_dict)

In [57]:
my_dict

{'  ': 225,
 'A 1 11': 174,
 'A 1 12': 197,
 'A 1 14': 188,
 'A 2 20': 171,
 'A 3 31': 199,
 'A 3 32': 179,
 'B 5 51': 223,
 'B 6 61': 209,
 'B 6 62': 207,
 'B 7 71': 121,
 'B 7 72': 80,
 'B 8 80': 218,
 'C 10 101': 97,
 'C 10 102': 10,
 'C 10 103': 88,
 'C 10 104': 100,
 'C 10 105': 135,
 'C 10 106': 72,
 'C 10 107': 7,
 'C 10 108': 178,
 'C 11 111': 148,
 'C 11 112': 110,
 'C 12 120': 224,
 'C 13 131': 158,
 'C 13 132': 107,
 'C 13 133': 177,
 'C 13 134': 103,
 'C 13 139': 111,
 'C 14 141': 43,
 'C 14 142': 215,
 'C 14 143': 180,
 'C 14 144': 122,
 'C 15 151': 91,
 'C 15 152': 182,
 'C 16 161': 95,
 'C 16 162': 115,
 'C 16 163': 67,
 'C 17 171': 108,
 'C 17 172': 138,
 'C 17 179': 129,
 'C 18 181': 81,
 'C 18 182': 204,
 'C 19 191': 195,
 'C 19 192': 203,
 'C 20 201': 151,
 'C 20 202': 90,
 'C 20 203': 56,
 'C 20 204': 79,
 'C 20 205': 157,
 'C 21 211': 206,
 'C 21 212': 187,
 'C 21 213': 201,
 'C 22 221': 18,
 'C 22 222': 33,
 'C 23 231': 156,
 'C 23 232': 120,
 'C 23 233': 94,
 'C 

In [14]:
train[train['text'].isnull()]

,text,target


In [15]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [16]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220404_inference/model' + str(fold), save_function=accelerator.save)

In [17]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  my_file = Path("/content/drive/MyDrive/220404_inference/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 4
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

...

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

epoch 1 train acc 0.69893
epoch 1 train acc 0.70008
epoch 1 train acc 0.70028
epoch 1 train acc 0.69842
epoch 1 train acc 0.70085
epoch 1 train acc 0.70025
epoch 1 train acc 0.69836
epoch 1 train acc 0.70138
epoch 1 test acc 0.8651294757033248
epoch 1 test acc 0.8646898976982097
epoch 1 test acc 0.8628916240409207
epoch 1 test acc 0.8645300511508951
epoch 1 test acc 0.8648497442455243
epoch 1 test acc 0.8641304347826086
epoch 1 test acc 0.8675671355498721
epoch 1 test acc 0.8602541560102301
epoch 2 train acc 0.87847
epoch 2 train acc 0.87929
epoch 2 train acc 0.8784
epoch 2 train acc 0.87791
epoch 2 train acc 0.87774
epoch 2 train acc 0.87773
epoch 2 train acc 0.87727
epoch 2 train acc 0.87859
epoch 2 test acc 0.8957400895140665
epoch 2 test acc 0.8950207800511509
epoch 2 test acc 0.897818094629156
epoch 2 test acc 0.896019820971867
epoch 2 test acc 0.8949808184143222
epoch 2 test acc 0.8947410485933504
epoch 2 test acc 0.8951806265984654
epoch 2 test acc 0.8964194373401535
epoch 3 tra

# 추론

In [59]:
test = test.fillna('')

In [60]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_000001,,,,치킨전문점에서,고객의주문에의해,치킨판매,치킨전문점에서 고객의주문에의해 치킨판매,
1,id_000002,,,,산업공구,다른 소매업자에게,철물 수공구,산업공구 다른 소매업자에게 철물 수공구,
2,id_000003,,,,절에서,신도을 대상으로,불교단체운영,절에서 신도을 대상으로 불교단체운영,
3,id_000004,,,,영업장에서,고객요구로,자동차튜닝,영업장에서 고객요구로 자동차튜닝,
4,id_000005,,,,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공","실내포장마차에서 접객시설을 갖추고 소주,맥주제공",
...,...,...,...,...,...,...,...,...,...
99995,id_099996,,,,사업장에서,일반인대상으로,버섯농장,사업장에서 일반인대상으로 버섯농장,
99996,id_099997,,,,한의원에서,외래환자위주고,치료,한의원에서 외래환자위주고 치료,
99997,id_099998,,,,일반점포에서,소비자에게,그림판매,일반점포에서 소비자에게 그림판매,
99998,id_099999,,,,사업장에서,일반인.학생대상으로,학습공간제공,사업장에서 일반인.학생대상으로 학습공간제공,


In [61]:
test = test[['text', 'target']]
test

,text,target
0,치킨전문점에서 고객의주문에의해 치킨판매,
1,산업공구 다른 소매업자에게 철물 수공구,
2,절에서 신도을 대상으로 불교단체운영,
3,영업장에서 고객요구로 자동차튜닝,
4,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공",
...,...,...
99995,사업장에서 일반인대상으로 버섯농장,
99996,한의원에서 외래환자위주고 치료,
99997,일반점포에서 소비자에게 그림판매,
99998,사업장에서 일반인.학생대상으로 학습공간제공,


In [62]:
test['target'].loc[0]

'  '

In [63]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220404_inference/model' + str(fold), num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220404_inference/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220404_inference/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
print(my_variable)

In [27]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [44]:
pred0

,index,label
0,0,6
1,1,51
2,2,23
3,3,145
4,4,17
...,...,...
99995,99995,1
99996,99996,50
99997,99997,30
99998,99998,76


In [64]:
pred0 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_label4.csv')

In [65]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [66]:
df.head(3)

,text,target,label0,label1,label2,label3,label4
0,치킨전문점에서 고객의주문에의해 치킨판매,,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,산업공구 다른 소매업자에게 철물 수공구,,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,절에서 신도을 대상으로 불교단체운영,,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949


In [67]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

0.91629

In [68]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

,text,target,label0,label1,label2,label3,label4
3,영업장에서 고객요구로 자동차튜닝,,C 30 302,S 95 952,S 95 952,S 95 952,S 95 952
9,"약품(화공), 미싱 완성품입고, 수선",,C 29 292,C 25 259,C 25 259,C 25 259,C 27 271
17,"상가에서 주문에 의해 반찬,도시락",,C 10 107,C 10 107,C 10 107,G 47 472,C 10 107
21,매니큐어용기,,C 18 181,G 47 478,C 22 222,C 33 339,G 47 475
31,산업사용자에게 선별장에서 농산물도매,,G 46 463,G 46 462,G 46 463,G 46 462,G 46 462
...,...,...,...,...,...,...,...
99957,음식점에서 접객시설을 갖추고 곱창밑 주류판매,,I 56 562,I 56 562,I 56 561,I 56 561,I 56 561
99979,업주들을대상으로 사무실에서 검사및회계사무지원,,M 71 712,M 71 715,M 71 715,M 71 715,M 71 715
99982,"사무실에서 교육관련 강사섭외,마케팅",,P 85 857,P 85 857,P 85 857,P 85 856,P 85 857
99985,"철 구입, 기계절단",,C 25 259,C 25 259,C 25 259,C 29 291,C 25 259


In [69]:
pred0 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220404_inference/pred_prob4.csv')

In [70]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,215,216,217,218,219,220,221,222,223,224
0,치킨전문점에서 고객의주문에의해 치킨판매,,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,1.921639e-07,0.000213,3.196115e-07,...,2.556804e-08,1.238009e-08,1.682169e-08,9.681956e-09,9.205679e-10,4.360617e-09,5.835973e-09,2.374930e-08,2.075946e-09,8.424250e-09
1,산업공구 다른 소매업자에게 철물 수공구,,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,2.837635e-06,0.000006,1.105055e-03,...,3.141799e-07,7.410741e-08,1.031685e-06,1.631821e-07,2.070810e-08,4.997086e-08,8.406297e-08,3.730703e-07,1.282353e-08,8.334327e-08
2,절에서 신도을 대상으로 불교단체운영,,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,9.921188e-07,0.000003,1.612360e-06,...,6.514478e-09,3.237832e-08,4.592942e-09,1.184487e-08,8.690191e-10,1.802465e-08,7.263485e-09,6.958574e-09,4.918694e-10,1.602653e-09


In [71]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,216,217,218,219,220,221,222,223,224,soft
0,치킨전문점에서 고객의주문에의해 치킨판매,,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,1.921639e-07,0.000213,3.196115e-07,...,1.238009e-08,1.682169e-08,9.681956e-09,9.205679e-10,4.360617e-09,5.835973e-09,2.374930e-08,2.075946e-09,8.424250e-09,I 56 561
1,산업공구 다른 소매업자에게 철물 수공구,,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,2.837635e-06,0.000006,1.105055e-03,...,7.410741e-08,1.031685e-06,1.631821e-07,2.070810e-08,4.997086e-08,8.406297e-08,3.730703e-07,1.282353e-08,8.334327e-08,G 46 466
2,절에서 신도을 대상으로 불교단체운영,,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,9.921188e-07,0.000003,1.612360e-06,...,3.237832e-08,4.592942e-09,1.184487e-08,8.690191e-10,1.802465e-08,7.263485e-09,6.958574e-09,4.918694e-10,1.602653e-09,S 94 949


In [72]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

,label0,label1,label2,label3,label4,soft
0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
3,C 30 302,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952
4,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562
...,...,...,...,...,...,...
99995,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
99996,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862
99997,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
99998,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902


In [73]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [74]:
soft[soft['soft'] != soft['hard']]

,label0,label1,label2,label3,label4,soft,temp,hard
21,C 18 181,G 47 478,C 22 222,C 33 339,G 47 475,C 33 339,"{'C 18 181': 1, 'G 47 478': 1, 'C 22 222': 1, ...",0
44,C 25 259,C 33 339,G 46 464,G 46 464,C 20 204,G 46 464,"{'C 25 259': 1, 'C 33 339': 1, 'G 46 464': 2, ...",0
85,G 45 452,G 45 452,S 95 952,G 45 452,S 95 952,S 95 952,"{'G 45 452': 3, 'S 95 952': 2}",G 45 452
91,G 46 465,G 46 466,G 46 466,G 46 465,G 46 467,G 46 465,"{'G 46 465': 2, 'G 46 466': 2, 'G 46 467': 1}",0
140,R 91 912,I 56 561,I 56 561,S 96 961,G 47 472,S 96 961,"{'R 91 912': 1, 'I 56 561': 2, 'S 96 961': 1, ...",0
...,...,...,...,...,...,...,...,...
99663,C 20 203,C 10 108,C 23 239,C 23 232,C 23 239,C 23 239,"{'C 20 203': 1, 'C 10 108': 1, 'C 23 239': 2, ...",0
99667,C 10 103,C 10 103,C 10 107,C 10 103,C 10 107,C 10 107,"{'C 10 103': 3, 'C 10 107': 2}",C 10 103
99743,M 70 702,S 94 941,O 84 842,M 70 702,O 84 842,O 84 842,"{'M 70 702': 2, 'S 94 941': 1, 'O 84 842': 2}",0
99770,G 46 466,G 46 466,G 46 466,G 46 465,G 46 465,G 46 465,"{'G 46 466': 3, 'G 46 465': 2}",G 46 466


In [75]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

,label0,label1,label2,label3,label4,soft,hard
0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
3,C 30 302,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952
4,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562
...,...,...,...,...,...,...,...
99995,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
99996,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862
99997,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
99998,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902


In [76]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [41]:
print(my_variable)

NameError: ignored

In [78]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [79]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [80]:
submission.to_csv('/content/drive/MyDrive/220404_inference/submission.csv', index=False)